# Segmenting and Clustering Neighborhoods in Toronto - Part 3

##### by Ahmed 

## Retrieving Toronto Neighborhood Data

In [1]:
import pandas as pd
import numpy as np

In [2]:
!pip install beautifulsoup4
!pip install lxml
!pip install html5lib
!pip install requests


spyder 3.3.4 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 3.3.4 requires pyqtwebengine<5.13, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
spyder 3.3.4 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 3.3.4 requires pyqtwebengine<5.13, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.
spyder 3.3.4 requires pyqt5<5.13; python_version >= "3", which is not installed.
spyder 3.3.4 requires pyqtwebengine<5.13, which is not installed.
distributed 1.21.8 requires msgpack, which is not installed.
You are using pip version 10.0.1, however version 19.1.1 is available.
You should co

See previous submissions for more insight into the data with outputs

In [3]:
from bs4 import BeautifulSoup
import requests

In [4]:
# Importing Webpage data with BeautifulSoup

source  = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text

soup = BeautifulSoup(source, 'lxml')

In [5]:
# Retrieving table from soup and organizing into a list
table = soup.find('table')
table_raw = table.tbody.text

raw_list = table_raw.splitlines()

In [6]:
# Removing empty space
i = 0

length = raw_list.count('')
while i < length:
    
    raw_list.remove('')
    i= i + 1

In [7]:
# Placing data from table into lists for 'Postcode','Borough', and 'Neighbourhood'

col_1_ = [] # Postcode
col_2_ = [] # Borough
col_3_ = [] # Neighbourhood

i = 3
while i < len(raw_list):
    col_1_.append(raw_list[i])
    col_2_.append(raw_list[i+1])
    col_3_.append(raw_list[i+2])
    i = i + 3

In [8]:
# Making a DataFrame with lists
data = {'Postcode':col_1_, 'Borough':col_2_, 'Neighbourhood':col_3_,}
df = pd.DataFrame(data)
df = df[['Postcode','Borough', 'Neighbourhood' ]]

In [9]:
# Retrieving indeces with "Not assigned" in the Borough colomn
index_na = df[df['Borough'].isin(['Not assigned'])].index
index_na = list(index_na)

# Dropping rows with "Not assigned" in the Borough colomn
df_2 = df
df_2 = df_2.drop(index_na)

In [10]:
# Replacing 'Not assigned' in Queen's Park Borough with the Borough's name 

df_2.loc[8,['Neighbourhood']] = "Queen's Park"

In [11]:
# Grouping Neighbourhoods in the same Postcode

df3 = df_2.groupby(['Postcode','Borough'], sort = False).agg(lambda x: ', '.join(x))

In [12]:
# Main DataFrame with index reset
df = df3.reset_index()
df.head()

,Postcode,Borough,Neighbourhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Harbourfront, Regent Park"
3,M6A,North York,"Lawrence Heights, Lawrence Manor"
4,M7A,Queen's Park,Queen's Park


In [13]:
df.shape

(103, 3)

In [14]:
# Adding .csv file provided for Longitude and Latitude

df_data_1 = pd.read_csv('/Users/ahmedmashhour/Downloads/Geospatial_Coordinates.csv')
df_data_1.head(5)

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [15]:
df_e = df

In [16]:
# Merging Latitude and Longitude from df_data_1 to df

df = df_e.merge(df_data_1, left_on = 'Postcode', right_on = 'Postal Code')
del df['Postal Code']
df.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
3,M6A,North York,"Lawrence Heights, Lawrence Manor",43.718518,-79.464763
4,M7A,Queen's Park,Queen's Park,43.662301,-79.389494
5,M9A,Etobicoke,Islington Avenue,43.667856,-79.532242
6,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
7,M3B,North York,Don Mills North,43.745906,-79.352188
8,M4B,East York,"Woodbine Gardens, Parkview Hill",43.706397,-79.309937
9,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937


# Main Analysis On Neighborhoods

## Exploring Toronto

###### The neighborhoods will belong only to the boroughs containing the word Toronto

In [17]:
# Main DataFrame used for Analysis

df_main = df[df['Borough'].str.contains('Toronto')].reset_index(drop = True)
df_main.head(10)

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
3,M4E,East Toronto,The Beaches,43.676357,-79.293031
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306
5,M5G,Downtown Toronto,Central Bay Street,43.657952,-79.387383
6,M6G,Downtown Toronto,Christie,43.669542,-79.422564
7,M5H,Downtown Toronto,"Adelaide, King, Richmond",43.650571,-79.384568
8,M6H,West Toronto,"Dovercourt Village, Dufferin",43.669005,-79.442259
9,M5J,Downtown Toronto,"Harbourfront East, Toronto Islands, Union Station",43.640816,-79.381752


Getting Toronto Latitude and Longitude

In [18]:
# Random location in Downtown from dataframe

Latitude = list(df_main.Latitude)[0]  
Longitude = list(df_main.Longitude)[0] 


Mapping neighborhood locations 

In [19]:
import folium

map_toronto = folium.Map(location=[Latitude, Longitude], zoom_start=12)

for lat, lng, label in zip(df_main['Latitude'], df_main['Longitude'], df_main['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  

map_toronto

Getting nearby neighborhood venues using a function with an API request from FOURSQUARE

In [20]:
CLIENT_ID = 'removed on github' # My Foursquare ID
CLIENT_SECRET = 'removed on github' # My Foursquare Secret
VERSION = '20191305' # Foursquare API version

LIMIT = 100 
radius = 500

In [21]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [22]:
toronto_venues = getNearbyVenues(names=df_main['Neighbourhood'],
                                   latitudes=df_main['Latitude'],
                                   longitudes=df_main['Longitude']
                                  )


Harbourfront, Regent Park
Ryerson, Garden District
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Adelaide, King, Richmond
Dovercourt Village, Dufferin
Harbourfront East, Toronto Islands, Union Station
Little Portugal, Trinity
The Danforth West, Riverdale
Design Exchange, Toronto Dominion Centre
Brockton, Exhibition Place, Parkdale Village
The Beaches West, India Bazaar
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North, Forest Hill West
High Park, The Junction South
North Toronto West
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
Harbord, University of Toronto
Runnymede, Swansea
Moore Park, Summerhill East
Chinatown, Grange Park, Kensington Market
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Rosedale
Stn A PO Boxes 25 The Esplanade
Cabbagetown, St. James Town
Fir

In [23]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Harbourfront, Regent Park",43.65426,-79.360636,Roselle Desserts,43.653447,-79.362017,Bakery
1,"Harbourfront, Regent Park",43.65426,-79.360636,Tandem Coffee,43.653559,-79.361809,Coffee Shop
2,"Harbourfront, Regent Park",43.65426,-79.360636,Toronto Cooper Koo Family Cherry St YMCA Centre,43.653191,-79.357947,Gym / Fitness Center
3,"Harbourfront, Regent Park",43.65426,-79.360636,Body Blitz Spa East,43.654735,-79.359874,Spa
4,"Harbourfront, Regent Park",43.65426,-79.360636,Morning Glory Cafe,43.653947,-79.361149,Breakfast Spot


Checking how many venues where retured for group of neighbohoods

In [24]:
toronto_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,57,57,57,57,57,57
"Brockton, Exhibition Place, Parkdale Village",19,19,19,19,19,19
Business Reply Mail Processing Centre 969 Eastern,17,17,17,17,17,17
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",44,44,44,44,44,44
Central Bay Street,88,88,88,88,88,88
"Chinatown, Grange Park, Kensington Market",100,100,100,100,100,100
Christie,16,16,16,16,16,16


Finding unique venue categories

In [25]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))

There are 236 uniques categories.


## Analysing Neighborhoods

Grouping venues to get the top 5 for each neighborhood or group of neighborhoods

In [26]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

#toronto_onehot.head()

In [27]:
toronto_onehot.shape

(1700, 236)

Grouping rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [28]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Yoga Studio,Adult Boutique,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,...,Theme Restaurant,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint
0,"Adelaide, King, Richmond",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.010000,0.0,0.0,0.01,0.0
1,Berczy Park,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.017544,0.0,0.0,0.00,0.0
2,"Brockton, Exhibition Place, Parkdale Village",0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
3,Business Reply Mail Processing Centre 969 Eastern,0.058824,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.000000,0.0,0.0,0.066667,0.066667,0.066667,0.133333,0.2,0.133333,...,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0


In [29]:
toronto_grouped.shape

(38, 236)

#### Printing neighborhoods along with the top 5 most common venues

In [30]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
                 venue  freq
0          Coffee Shop  0.06
1                 Café  0.05
2      Thai Restaurant  0.04
3           Steakhouse  0.04
4  American Restaurant  0.04


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.05
2          Restaurant  0.04
3  Seafood Restaurant  0.04
4              Bakery  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.11
1            Café  0.11
2     Coffee Shop  0.11
3   Burrito Place  0.05
4             Bar  0.05


----Business Reply Mail Processing Centre 969 Eastern----
                venue  freq
0  Light Rail Station  0.12
1         Yoga Studio  0.06
2       Garden Center  0.06
3          Comic Shop  0.06
4                Park  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
              venue  freq
0   Airport Service  0.20
1    Ai

#### Converting to a dataframe

In [31]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [32]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,"Adelaide, King, Richmond",Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bar,Burger Joint,Gym,Hotel,Bakery
1,Berczy Park,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Farmers Market,Beer Bar,Steakhouse,Seafood Restaurant,Restaurant,Bakery
2,"Brockton, Exhibition Place, Parkdale Village",Café,Breakfast Spot,Coffee Shop,Grocery Store,Italian Restaurant,Caribbean Restaurant,Stadium,Bar,Furniture / Home Store,Burrito Place
3,Business Reply Mail Processing Centre 969 Eastern,Light Rail Station,Garden,Recording Studio,Auto Workshop,Skate Park,Burrito Place,Fast Food Restaurant,Farmers Market,Brewery,Restaurant
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",Airport Service,Airport Lounge,Airport Terminal,Harbor / Marina,Sculpture Garden,Boutique,Plane,Boat or Ferry,Airport Gate,Airport


## Clustering Neighborhoods

We'll use 4 clusters for Toronto

In [33]:
# set number of clusters
from sklearn.cluster import KMeans

kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:20] 

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int32)

In [34]:
print(kmeans.labels_)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 3 1 1 0 2 1 1 1 1 1 1 1 1
 1]


Re-contructing the main dataframe along with the cluster labels and most common venues

In [35]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = df_main

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')

toronto_merged.head() # check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M5A,Downtown Toronto,"Harbourfront, Regent Park",43.654260,-79.360636,1,Coffee Shop,Park,Café,Pub,Bakery,Mexican Restaurant,Breakfast Spot,Theater,Chocolate Shop,Beer Store
1,M5B,Downtown Toronto,"Ryerson, Garden District",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Tea Room,Pizza Place,Sporting Goods Shop,Japanese Restaurant,Fast Food Restaurant
2,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418,1,Coffee Shop,Hotel,Restaurant,Café,Cosmetics Shop,Breakfast Spot,Gastropub,Bakery,Italian Restaurant,Diner
3,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Health Food Store,Pub,Music Venue,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
4,M5E,Downtown Toronto,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Farmers Market,Beer Bar,Steakhouse,Seafood Restaurant,Restaurant,Bakery


Visualizing the clusters in Toronto

In [36]:
import matplotlib.cm as cm
import matplotlib.colors as colors



# create map
map_clusters = folium.Map(location=[Latitude, Longitude], zoom_start=11.3)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters


Observations: It looks like Toronto neighborhood venues are not as diverse in attributes an NYC 

### Examining Clusters

In [37]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Central Toronto,0,Bus Line,Park,Construction & Landscaping,Swim School,Wings Joint,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
32,Downtown Toronto,0,Park,Playground,Trail,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


In [38]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Downtown Toronto,1,Coffee Shop,Park,Café,Pub,Bakery,Mexican Restaurant,Breakfast Spot,Theater,Chocolate Shop,Beer Store
1,Downtown Toronto,1,Coffee Shop,Clothing Store,Cosmetics Shop,Café,Middle Eastern Restaurant,Tea Room,Pizza Place,Sporting Goods Shop,Japanese Restaurant,Fast Food Restaurant
2,Downtown Toronto,1,Coffee Shop,Hotel,Restaurant,Café,Cosmetics Shop,Breakfast Spot,Gastropub,Bakery,Italian Restaurant,Diner
3,East Toronto,1,Health Food Store,Pub,Music Venue,Wings Joint,Department Store,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
4,Downtown Toronto,1,Coffee Shop,Cocktail Bar,Café,Cheese Shop,Farmers Market,Beer Bar,Steakhouse,Seafood Restaurant,Restaurant,Bakery
5,Downtown Toronto,1,Coffee Shop,Café,Italian Restaurant,Burger Joint,Chinese Restaurant,Japanese Restaurant,Restaurant,Sandwich Place,Bubble Tea Shop,Bar
6,Downtown Toronto,1,Grocery Store,Café,Park,Restaurant,Nightclub,Athletics & Sports,Baby Store,Diner,Italian Restaurant,Convenience Store
7,Downtown Toronto,1,Coffee Shop,Café,Thai Restaurant,Steakhouse,American Restaurant,Bar,Burger Joint,Gym,Hotel,Bakery
8,West Toronto,1,Supermarket,Bakery,Pharmacy,Music Venue,Middle Eastern Restaurant,Bank,Bar,Discount Store,Pool,Café
9,Downtown Toronto,1,Coffee Shop,Hotel,Aquarium,Italian Restaurant,Café,Fried Chicken Joint,Restaurant,Scenic Lookout,Pizza Place,Bakery


In [39]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
18,Central Toronto,2,Garden,Wings Joint,Department Store,Event Space,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant


In [40]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
28,Central Toronto,3,Playground,Trail,Tennis Court,Wings Joint,Dog Run,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Donut Shop
